In [1]:
import pandas as pd
import geopandas as gpd

In [2]:
df = pd.read_csv('./datasets/DOH COVID Data Drop_20220917/DOH COVID Data Drop_ 20220917 - 04 Case Information_batch_3.csv')
df.head()

C:\Users\flipp\AppData\Local\Temp\ipykernel_12052\3966465681.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./datasets/DOH COVID Data Drop_20220917/DOH COVID Data Drop_ 20220917 - 04 Case Information_batch_3.csv')


,CaseCode,Age,AgeGroup,Sex,DateSpecimen,DateResultRelease,DateRepConf,DateDied,DateRecover,RemovalType,...,ProvRes,CityMunRes,CityMuniPSGC,BarangayRes,BarangayPSGC,HealthStatus,Quarantined,DateOnset,Pregnanttab,ValidationStatus
0,C79128876,40.0,40 to 44,MALE,2022-01-07,2022-01-10,2022-01-11,NaN,NaN,RECOVERED,...,NCR,CITY OF PARAÑAQUE,PH137604000,SAN ISIDRO,PH137604013,RECOVERED,NO,NaN,NaN,"Removal Type is ""Recovered"", but no Recovered ..."
1,C94164354,40.0,40 to 44,FEMALE,2022-01-08,2022-01-09,2022-01-11,NaN,NaN,RECOVERED,...,RIZAL,SAN MATEO,PH045811000,GUITNANG BAYAN II (POB.),PH045811006,RECOVERED,NO,2022-01-05,NO,"Health Status is ""Recovered"", but no Date Reco..."
2,C1733397,41.0,40 to 44,MALE,2022-01-06,2022-01-07,2022-01-11,NaN,NaN,RECOVERED,...,NCR,CITY OF PASIG,PH137403000,KAPITOLYO,PH137403009,RECOVERED,NO,NaN,NaN,"Removal Type is ""Recovered"", but no Recovered ..."
3,C95771390,50.0,50 to 54,FEMALE,2022-01-07,2022-01-09,2022-01-11,NaN,NaN,RECOVERED,...,NCR,CALOOCAN CITY,PH137501000,BARANGAY 100,PH137501100,RECOVERED,NO,NaN,NO,"Removal Type is ""Recovered"", but no Recovered ..."
4,C44529762,26.0,25 to 29,FEMALE,NaN,2022-01-07,2022-01-11,NaN,NaN,RECOVERED,...,NCR,CALOOCAN CITY,PH137501000,BARANGAY 178,PH137501178,RECOVERED,NO,NaN,NO,"Removal Type is ""Recovered"", but no Recovered ..."


In [3]:
len(df[df.HealthStatus != 'RECOVERED']) + len(df[df.HealthStatus != 'DIED'])

944600

In [4]:
df_active_cases = df[(df.HealthStatus != 'RECOVERED') & (df.HealthStatus != 'DIED')].groupby('BarangayPSGC').agg(active_cases = ('BarangayPSGC', 'count')).sort_values('active_cases', ascending=False)
df_active_cases

,active_cases
BarangayPSGC,
PH137605076,200
PH137607020,186
PH045805015,108
PH137404139,105
PH137604007,102
...,...
PH051737003,1
PH051736022,1
PH051736015,1


In [5]:
gdf = gpd.read_file('./datasets/Barangays/Barangays.shp')

gdf.head()

,ADM1_PCODE,ADM2_PCODE,ADM3_PCODE,ADM4_PCODE,ADM4_EN,ADM3_EN,ADM2_EN,ADM1_EN,ADM_ID,UPDATED,PERIMETER,AREA,LONGITUDE,LATITUDE,geometry
0,PH010000000,PH015500000,PH015502000,PH015502012,Pogomboa,Aguilar,Pangasinan,Region I,15502012.0,2017-12-31,5590.506,1.745360e+06,120.243,15.882,"POLYGON ((120.25441 15.88112, 120.24723 15.878..."
1,PH010000000,PH015500000,PH015502000,PH015502013,Pogonsili,Aguilar,Pangasinan,Region I,15502013.0,2017-12-31,30658.516,2.011989e+07,120.215,15.791,"POLYGON ((120.26372 15.81724, 120.25821 15.811..."
2,PH010000000,PH015500000,PH015502000,PH015502014,San Jose,Aguilar,Pangasinan,Region I,15502014.0,2017-12-31,13404.869,5.289032e+06,120.280,15.867,"POLYGON ((120.28018 15.87048, 120.27885 15.870..."
3,PH010000000,PH015500000,PH015502000,PH015502016,Tampac,Aguilar,Pangasinan,Region I,15502016.0,2017-12-31,6235.011,1.721892e+06,120.251,15.875,"POLYGON ((120.26163 15.87600, 120.26149 15.875..."
4,PH010000000,PH015500000,PH015502000,PH015502017,Laoag,Aguilar,Pangasinan,Region I,15502017.0,2017-12-31,36419.270,3.944797e+07,120.191,15.856,"POLYGON ((120.22100 15.88999, 120.22163 15.886..."


In [6]:
df_active_cases = gpd.GeoDataFrame(df_active_cases)

In [7]:
merged = gpd.GeoDataFrame.merge(df_active_cases, gdf, how='right', left_on='BarangayPSGC', right_on='ADM4_PCODE')

merged.set_crs('epsg:4326')

merged.fillna(0, inplace=True)

c:\Users\flipp\anaconda3\lib\site-packages\geopandas\geodataframe.py:1503: FutureWarning: Accessing CRS of a GeoDataFrame without a geometry column is deprecated and will be removed in GeoPandas 0.12. Use GeoDataFrame.set_geometry to set the active geometry column.
  result.crs = self.crs


In [8]:
merged[['active_cases', 'ADM4_EN', 'ADM3_EN']].sort_values('active_cases', ascending=False)

,active_cases,ADM4_EN,ADM3_EN
34226,200.0,Barangay 76,Pasay City
34390,186.0,Fort Bonifacio,Taguig City
12598,108.0,San Isidro,Cainta
33793,105.0,Batasan Hills,Quezon City
34141,102.0,B. F. Homes,City of Parañaque
...,...,...,...
18780,0.0,Tacuyong Sur,Leon
18779,0.0,Tacuyong Norte,Leon
18778,0.0,Marirong,Leon
18777,0.0,Manampunay,Leon


In [9]:
merged.head()

,active_cases,ADM1_PCODE,ADM2_PCODE,ADM3_PCODE,ADM4_PCODE,ADM4_EN,ADM3_EN,ADM2_EN,ADM1_EN,ADM_ID,UPDATED,PERIMETER,AREA,LONGITUDE,LATITUDE,geometry
0,1.0,PH010000000,PH015500000,PH015502000,PH015502012,Pogomboa,Aguilar,Pangasinan,Region I,15502012.0,2017-12-31,5590.506,1.745360e+06,120.243,15.882,"POLYGON ((120.25441 15.88112, 120.24723 15.878..."
1,0.0,PH010000000,PH015500000,PH015502000,PH015502013,Pogonsili,Aguilar,Pangasinan,Region I,15502013.0,2017-12-31,30658.516,2.011989e+07,120.215,15.791,"POLYGON ((120.26372 15.81724, 120.25821 15.811..."
2,0.0,PH010000000,PH015500000,PH015502000,PH015502014,San Jose,Aguilar,Pangasinan,Region I,15502014.0,2017-12-31,13404.869,5.289032e+06,120.280,15.867,"POLYGON ((120.28018 15.87048, 120.27885 15.870..."
3,0.0,PH010000000,PH015500000,PH015502000,PH015502016,Tampac,Aguilar,Pangasinan,Region I,15502016.0,2017-12-31,6235.011,1.721892e+06,120.251,15.875,"POLYGON ((120.26163 15.87600, 120.26149 15.875..."
4,0.0,PH010000000,PH015500000,PH015502000,PH015502017,Laoag,Aguilar,Pangasinan,Region I,15502017.0,2017-12-31,36419.270,3.944797e+07,120.191,15.856,"POLYGON ((120.22100 15.88999, 120.22163 15.886..."
